In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [7]:
from io import StringIO


"""
Date	Rice	Wheat	Atta (Wheat)	Gram Dal
01-01-2016	29.5	30.5	26.67	70.5
02-01-2016	29.6	31.4	27.5	71.4
03-01-2016	29.6	31.4	27.5	71.4
04-01-2016	29.6	31.4	27.5	71.4
05-01-2016	29.5	31.75	27.5	71.75
06-01-2016	29.4	31.2	27.5	70.6
07-01-2016	29.4	31.2	27.5	70.6
08-01-2016	29.4	31.2	27.5	70.2
09-01-2016	29.25	31.5	27.5	70.25
10-01-2016	29.4	31.2	27.5	70.2
"""
data = """
Date,Rice,Wheat,Atta (Wheat),Gram Dal,sno
01-01-2016,29.5,30.5,26.67,70.5,1
02-01-2016,29.6,31.4,27.5,71.4,2
03-01-2016,29.6,31.4,27.5,71.4,3
04-01-2016,29.6,31.4,27.5,71.4,4
05-01-2016,29.5,31.75,27.5,71.75,5
06-01-2016,29.4,31.2,27.5,70.6,6
07-01-2016,29.4,31.2,27.5,70.6,7
08-01-2016,29.4,31.2,27.5,70.2,8
09-01-2016,29.25,31.5,27.5,70.25,9
10-01-2016,29.4,31.2,27.5,70.2,10
"""

df1 = pd.read_csv(StringIO(data))
print(df1)

         Date   Rice  Wheat  Atta (Wheat)  Gram Dal  sno
0  01-01-2016  29.50  30.50         26.67     70.50    1
1  02-01-2016  29.60  31.40         27.50     71.40    2
2  03-01-2016  29.60  31.40         27.50     71.40    3
3  04-01-2016  29.60  31.40         27.50     71.40    4
4  05-01-2016  29.50  31.75         27.50     71.75    5
5  06-01-2016  29.40  31.20         27.50     70.60    6
6  07-01-2016  29.40  31.20         27.50     70.60    7
7  08-01-2016  29.40  31.20         27.50     70.20    8
8  09-01-2016  29.25  31.50         27.50     70.25    9
9  10-01-2016  29.40  31.20         27.50     70.20   10


In [2]:
df2 = pd.read_csv(r"data\cleansing\filled\mean_filled.csv")

In [3]:
df2.head()

,Date,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
0,2016-01-01,29.5,30.50,26.67,70.50,155.0,160.0,105.75,90.0,32.75,...,79.25,88.0,92.25,59.25,43.00,215.0,16.25,20.50,23.25,29.5
1,2016-01-02,29.6,31.40,27.50,71.40,154.6,161.2,108.20,92.5,33.00,...,82.00,88.0,92.00,59.00,45.00,216.0,16.60,20.40,23.60,32.0
2,2016-01-03,29.6,31.40,27.50,71.40,154.0,161.2,108.20,92.5,33.00,...,82.00,88.0,92.00,59.00,45.00,216.0,16.60,20.40,23.60,35.0
3,2016-01-04,29.6,31.40,27.50,71.40,151.6,160.2,107.20,90.5,34.40,...,82.00,88.0,92.00,59.00,45.00,216.0,16.60,21.40,23.00,41.4
4,2016-01-05,29.5,31.75,27.50,71.75,151.5,161.5,107.50,90.5,34.25,...,80.25,88.0,91.75,58.75,45.75,215.0,17.00,21.25,22.75,41.0


In [5]:
df2 ['Date']= pd.to_datetime(df2['Date'])
df2

,Date,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
0,2016-01-01,29.50,30.50,26.67,70.50,155.00,160.00,105.75,90.00,32.75,...,79.25,88.000000,92.25,59.25,43.00,215.00,16.25,20.50,23.25,29.50
1,2016-01-02,29.60,31.40,27.50,71.40,154.60,161.20,108.20,92.50,33.00,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,32.00
2,2016-01-03,29.60,31.40,27.50,71.40,154.00,161.20,108.20,92.50,33.00,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,20.40,23.60,35.00
3,2016-01-04,29.60,31.40,27.50,71.40,151.60,160.20,107.20,90.50,34.40,...,82.00,88.000000,92.00,59.00,45.00,216.00,16.60,21.40,23.00,41.40
4,2016-01-05,29.50,31.75,27.50,71.75,151.50,161.50,107.50,90.50,34.25,...,80.25,88.000000,91.75,58.75,45.75,215.00,17.00,21.25,22.75,41.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166,2024-09-01,56.12,44.88,52.88,99.76,172.82,137.65,120.76,89.47,44.18,...,126.29,120.706538,115.82,95.59,57.53,286.23,24.41,49.00,53.47,23.76
3167,2024-09-02,57.31,45.73,54.06,99.75,173.53,139.00,122.47,92.17,44.28,...,126.69,125.605161,116.00,95.78,57.19,283.88,24.16,48.97,53.84,23.94
3168,2024-09-03,57.53,45.80,54.75,99.88,172.23,138.19,121.97,91.96,44.22,...,126.38,125.397419,116.06,95.91,57.34,283.96,24.19,48.88,54.41,24.53
3169,2024-09-04,57.29,45.79,55.19,99.48,172.30,138.90,122.03,92.26,44.19,...,127.16,125.459032,116.16,95.97,56.84,287.13,24.07,49.32,55.45,26.58


In [6]:
df2.iloc[1,0].month

1

In [8]:
df1 ['Date']= pd.to_datetime(df1['Date'])
df1

,Date,Rice,Wheat,Atta (Wheat),Gram Dal,sno
0,2016-01-01,29.50,30.50,26.67,70.50,1
1,2016-02-01,29.60,31.40,27.50,71.40,2
2,2016-03-01,29.60,31.40,27.50,71.40,3
3,2016-04-01,29.60,31.40,27.50,71.40,4
4,2016-05-01,29.50,31.75,27.50,71.75,5
5,2016-06-01,29.40,31.20,27.50,70.60,6
6,2016-07-01,29.40,31.20,27.50,70.60,7
7,2016-08-01,29.40,31.20,27.50,70.20,8
8,2016-09-01,29.25,31.50,27.50,70.25,9
9,2016-10-01,29.40,31.20,27.50,70.20,10


In [9]:
df2.iloc[1,0].month


1

In [10]:
i=1
if (i==1):
    start_date = df2['Date'].min()
    end_date = df2['Date'].max()
    print(f"Start date:{start_date}, End date:{end_date}")
    

Start date:2016-01-01 00:00:00, End date:2024-09-05 00:00:00


In [32]:
# Print the date format like yyyy-mm-dd
df1['Date'] = df1['Date'].dt.strftime('%Y-%m-%d')
df1

,Date,Rice,Wheat,Atta (Wheat),Gram Dal,sno
0,2016-01-01,29.50,30.50,26.67,70.50,1
1,2016-02-01,29.60,31.40,27.50,71.40,2
2,2016-03-01,29.60,31.40,27.50,71.40,3
3,2016-04-01,29.60,31.40,27.50,71.40,4
4,2016-05-01,29.50,31.75,27.50,71.75,5
5,2016-06-01,29.40,31.20,27.50,70.60,6
6,2016-07-01,29.40,31.20,27.50,70.60,7
7,2016-08-01,29.40,31.20,27.50,70.20,8
8,2016-09-01,29.25,31.50,27.50,70.25,9
9,2016-10-01,29.40,31.20,27.50,70.20,10


In [11]:
dataset=df2

In [12]:
start_date = dataset['Date'].min()
end_date = dataset['Date'].max()
print(f"Start date:{start_date}, End date:{end_date}")

Start date:2016-01-01 00:00:00, End date:2024-09-05 00:00:00


In [16]:
date='2024-06-01'
print(date)

2024-06-01


In [17]:
dt=pd.to_datetime(date)
dt.month

6

In [ ]:
to_date = pd.to_datetime(date)
from_date = to_date - pd.DateOffset(days=look_back-1)

In [35]:

scalar = MinMaxScaler()
scalar.fit_transform(dataset)

ValueError: could not convert string to float: '2016-01-01'